# Vector Data

<p> Vector data are data in which geographic features are represented as discrete geometries—specifically, points, lines, and polygons. However, continuous data, such as elevation, cannot be considered as vector data </p>

### OGR Library

The OGR Simple Features Library is part of the Geospatial Data Abstraction Library (GDAL), an extremely popular open source library for reading and writing spatial data. The OGR portion of GDAL is the part that provides the ability to read and write many different vector data formats. OGR also allows you to create and manipulate geometries; edit attribute values; filter vector data based on attribute values or spatial location; and it also offers data analysis capabilities.

<img src="src.png"/>

### Import convention for OGR :

In [4]:
from osgeo import ogr

ModuleNotFoundError: No module named 'osgeo'

### Drivers
Get a driver (ESRI Shapefile):

In [6]:
driver = ogr.GetDriverByName('ESRI shapefile')
driver.name

NameError: name 'ogr' is not defined

### Open and iterate over a shapefile

In [ ]:
shapefile = '...'
ds = ogr.Open(shapefile, 0)
if ds is None:
    sys.exit('Error opening file')
ds

###  Get Layers of a shapefie

layer = ds.GetLayer(0)
layerName = layer.GetName()
layerName

In [ ]:
layer.GetFeatureCount()

Layer Extent

In [ ]:
ext = layer.GetExtent()
print(ext)
print('Upper-Left x:{}, y:{}'.format(ext[0], ext[3]))
print('Lower-Right x:{}, y:{}'.format(ext[1], ext[2]))

Attributes of layer

In [ ]:
[(fld.GetName(), ogr.GetFieldTypeName(fld.GetType()), fld.GetWidth()) for fld in layer.schema]

Iterate over the features of layer and work with geometry and values of fields

In [ ]:
chosenField = ''
for feature in layer:
    value = feat.GetField(chosenField)
    geom = feat.geometry()
    area = geom.Area()
    wkt = geom.ExportToWkt()
    print(area,wkt,value)
    break
layer.ResetReading()
ds = None

### Adding values to a shapefile

Adding new features to existing layers is exactly the same as adding them to empty layers. Create an empty feature based on the layer definition, populate it, and insert it into the layer.

In [9]:
layer.CreateField(ogr.FieldDefn('ID', ogr.OFTInteger))
uniqueID = 1
for feature in layer:
    feature.SetField('ID', uniqueID)
    layer.SetFeature(feature)
    uniqueID += 1

NameError: name 'layer' is not defined

## Raster Data

Raster data is a two- or threedimensional array of data values such as the pixels in a photograph. Raster datasets have the potential to include any type of data. Nonetheless, objects that can be thought of as points, lines, or polygons are usually better left as vectors. For example, country boundaries lend themselves perfectly to a polygon vector dataset. This same data could be stored in a raster, but it would take up more disk space and the boundaries wouldn’t be nice, smooth lines. You also couldn’t use vector data analysis functions such as buffering and intersecting. Raster is a perfect choice when values change continuously instead of at sharply defined boundaries. This includes common datasets such as elevation, slope, aspect, precipitation, temperature, and satellite data, but it can include many other things, too.

### GDAL Library

In [ ]:
from osgeo import gdal

### Translate netCDF to GeoTiff

In [ ]:
input_filename = "test.nc"
src_ds = gdal.Open(input_filename)
out_format_r = "GTiff"
driver_r = gdal.GetDriverByName(out_format_r)
out_rasterfile = "test.tif"
dst_ds = driver_r.CreateCopy(out_rasterfile, src_ds)

### Read Raster Files

In [ ]:
image = ''
ds = gdal.Open(image)

### Get image extent

In [ ]:
gt = ds.GetGeoTransform()
xmin,xmax,ymin,ymax = (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize,gt[3] + gt[5] * ds.RasterYSize)

### Calculate Dimensions

In [ ]:
rows = math.ceil((ymax - ymin) / -gt[5])
columns = math.ceil((xmax - xmin) / gt[1])

### Read Data of Raster

In [ ]:
data = ds.GetRasterBand(1).ReadAsArray()
data[2:4,6:8]

### Use Case: NDVI Calculation

In [11]:
import numpy as np

In [ ]:
red_ds = gdal.Open('B04.tif')
nir_ds = gdal.Open('B08.tif')
shape = red_ds.GetRasterBand(1).shape
red = red_ds.GetRasterBand(1).ReadAsArray().astype(np.float32)
nir = nir_ds.GetRasterBand(1).ReadAsArray().astype(np.float32)

check = np.logical_or(red > 0, nir > 0)
ndvi = np.where(check, (nir - red) / (nir + red), -999)

print("Total number of vegetation pixels:",np.count_nonzero(ndvi>0.2))

del check
del red
del nir

Save NDVI data as a raster

In [ ]:
geo = red_ds.GetGeoTransform()
proj = red_ds.GetProjection()
driver = gdal.GetDriverByName("GTiff")
dst_ds = driver.Create('ndvi.tif'), shape[1],shape[0], 1, gdal.GDT_Float32)
dst_ds.SetGeoTransform(geo)
dst_ds.SetProjection(proj)
dst_ds.GetRasterBand(1).WriteArray(ndvi)

del red_ds
del nir_ds
del dst_ds
del ndvi